In [8]:
from fastembed import TextEmbedding
import numpy as np

In [11]:
model_name = 'jinaai/jina-embeddings-v2-small-en'
embedder = TextEmbedding(model_name=model_name)

In [12]:
query = 'I just discovered the course. Can I join now?'
query_embedding = list(embedder.embed(query))[0]

print(query_embedding.shape)
print(query_embedding.min())

(512,)
-0.11726373885183883


In [13]:
doc = 'Can I still join the course after the start date?'
doc_embedding = list(embedder.embed(doc))[0]

print(np.dot(query_embedding, doc_embedding))

0.9008528895674548


In [14]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [17]:
docs_embeddings = list(embedder.embed([d["text"] for d in documents]))

docs_similarities = np.array([
    np.dot(query_embedding, d_e) for d_e in docs_embeddings
])

np.argmax(docs_similarities)


np.int64(1)

In [18]:
docs_embeddings = list(embedder.embed([
    d['question'] + ' ' + d['text'] for d in documents
]))

docs_similarities = np.array([
    np.dot(query_embedding, d_e) for d_e in docs_embeddings
])

np.argmax(docs_similarities)


np.int64(0)

In [21]:
min_fastembed_dim = min(model['dim'] for model in TextEmbedding.list_supported_models())
print(min_fastembed_dim)

384


In [27]:
small_model_name = 'BAAI/bge-small-en'
small_model_info = next(model for model in TextEmbedding.list_supported_models() if model['model'] == small_model_name)
assert small_model_info['dim'] == 384

In [23]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [29]:
import qdrant_client
from qdrant_client import models as qd_models

In [26]:
qd_client = qdrant_client.QdrantClient('http://localhost:6333')

In [30]:
collection_name = 'ml-zoomcamp-rag'

qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=qd_models.VectorParams(
        size=small_model_info['dim'],
        distance=qd_models.Distance.COSINE
    )
)

True

In [31]:
import uuid

points = []
for doc in documents:
    text = doc['question'] + ' ' + doc['text']
    
    point = qd_models.PointStruct(
        id=str(uuid.uuid4()),
        vector=qd_models.Document(text=text, model=small_model_name),
        payload=doc
    )
    points.append(point)    

In [32]:
qd_client.upsert(collection_name=collection_name, points=points)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [33]:
def search(query, limit=1):
    results = qd_client.query_points(
        collection_name=collection_name,
        query=qd_models.Document(text=query, model=small_model_name),
        limit=limit,
        with_payload=True,
    )

    return results

In [35]:
response = search(query)

In [36]:
response.points[0]

ScoredPoint(id='fe513dc7-ffb0-47b1-a2cc-ce6a6d27637e', version=0, score=0.8703172, payload={'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None)